# DAMEX implementation
_Mathis Hardion_

In [1]:
# Imports
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.datasets import fetch_covtype
from sklearn.utils import shuffle

# Set rng
rng = np.random.default_rng(seed=0)

In [2]:
# Helper to make boolean arrays hashable
def binary_to_str(alpha):
    return "".join(alpha.astype(int).astype(str))

In [3]:
# Dictionary to store only nonzero components of the estimator
class M_hat:
    def __init__(self):
        self._dict = {}

    def __setitem__(self, alpha, val):
        self._dict[binary_to_str(alpha)] = val

    def __getitem__(self, alpha):
        try:
            return self._dict[binary_to_str(alpha)]
        except KeyError:
            return 0

    def __len__(self):
        return len(self._dict)

    def avg_dim(self):
        keys = self._dict.keys()
        return sum(k.count('1') for k in keys)/len(keys)
    
    def apply_thresholding(self, threshold):
        for key in list(self._dict.keys()):
                if self._dict[key] < threshold:
                    del self._dict[key]

In [4]:
# Damex pipeline in a class akin to sklearn estimators
class DAMEX:
    def __init__(self, eps=0.01, k_fn=lambda n: int(np.sqrt(n)), p=0.1):
        # Set parameters
        self.eps = eps
        self.k_fn = k_fn
        self.p = p
        self.M = M_hat()
        self.data = None
        self.n = 0
        self.k = 0
        self.t = 0
        self.V = np.zeros(0)

    def rank_transform(self, X):
        # Transform data based on ecdf
        n, d = X.shape
        V = np.zeros((n, d))
        data = np.zeros((n, d))
        for j in range(d):
            idx = np.argsort(X[:,j])
            data[:,j] = X[idx, j]
            V[idx, j] = 1/(1-np.arange(n)/(n+1))
        return V, data
        
    def fit(self, X):
        n, d = X.shape
        self.n = n
        self.d = d
        self.data = np.zeros((n, d))
        # Rank transform
        V, data = self.rank_transform(X)
        self.data = np.vstack((data, np.inf*np.ones(d)))
        self.V = V
        # Estimation
        k = self.k_fn(n)
        self.k = k
        self.t = n/k
        Mtot = 0
        for i in range(n):
            alpha = V[i,:] > self.eps*self.t
            if alpha.sum():
                Mtot += 1/k
                self.M[alpha] = self.M[alpha] + 1/k
        # Thresholding
        card_A = len(self.M)
        threshold = self.p*Mtot/card_A
        self.M.apply_thresholding(threshold)
    
    def T(self, x):
        # Empirical transform based on training data
        idx = (x <= self.data).argmax(axis=0)
        v = 1/(1-idx/(self.n+1))
        return v
    
    def extreme_region(self, X):
        # Obtain extreme region of a set based on parameters
        V, _ = self.rank_transform(X)
        return V.max(axis=1) >= self.k
    def sn(self, x):
        # Scoring function
        v = self.T(x)
        alpha = v > self.eps*self.t
        return self.M[alpha]/v.max()

    def predict(self, X):
        return np.array([self.sn(x) for x in X])

In [18]:
# Get forestcover data
X, y = fetch_covtype(return_X_y=True, as_frame=True)
X, y = shuffle(X.to_numpy(), y.to_numpy())
X = X[(y == 2) | (y == 4)]
y = y[(y == 2) | (y == 4)]
y[y == 2] = 0
y[y == 4] = 1

In [19]:
X.shape

(286048, 54)

In [40]:
def repeat_experiments(X, y, estimators, N_exp, k_fn=lambda n: int(np.sqrt(n)), eps=.01):
    results = pd.DataFrame(columns=["ROC AUC", "PR AUC"], index=["DAMEX"] + [est.__class__.__name__ for est in estimators],
                          data=0.)
    anormal = y.astype(bool)
    X_normal = X[~anormal]
    y_normal = y[~anormal]
    n = len(y_normal)
    damex = DAMEX(k_fn=k_fn, eps=eps)
    avg_dim = 0
    for _ in range(N_exp):
        # Select training set among normal samples
        train = rng.binomial(1, .8, size=n).astype(bool)
        X_train = X_normal[train,:]
        y_train = y_normal[train]
        # Fit estimators
        damex.fit(X_train)
        avg_dim += damex.M.avg_dim()/N_exp
        for est in estimators:
            est.fit(X_train)
        # Test on extreme region only
        extreme = damex.extreme_region(X)
        test_normal = ~train & extreme[~anormal]
        test_anormal = extreme[anormal]
        X_test = np.vstack((X_normal[test_normal,:], X[anormal][test_anormal]))
        y_test = np.concatenate((y_normal[test_normal], y[anormal][test_anormal]))
        scores = damex.predict(X_test)
        results.loc["DAMEX", "ROC AUC"] += roc_auc_score(y_test, -scores)/N_exp
        results.loc["DAMEX", "PR AUC"] += average_precision_score(y_test, -scores)/N_exp
        for est in estimators:
            scores = est.decision_function(X_test)
            results.loc[est.__class__.__name__, "ROC AUC"] += roc_auc_score(y_test, -scores)/N_exp
            results.loc[est.__class__.__name__, "PR AUC"] += average_precision_score(y_test, -scores)/N_exp
    return results, avg_dim

In [48]:
# Only consider a subset of the data
N = 150000
# Hyperparameters
k_fn = lambda n: int(n**.25*np.log(n))
n = (.8*N)
d = X.shape[1]
eps = k_fn(n)**(1/3)*(1/d)**(4/3)
estimators = [IsolationForest(), LocalOutlierFactor(novelty=True)]
results, avg_dim = repeat_experiments(X[:80000], y[:80000], estimators, 10, k_fn=k_fn, eps=eps)

In [49]:
results

,ROC AUC,PR AUC
DAMEX,0.906492,0.639453
IsolationForest,0.889512,0.600632
LocalOutlierFactor,0.994338,0.980775


In [50]:
avg_dim

20.68608936087504